🛡️ Pydantic: The Ultimate Guide to Data Validation

Pydantic is a data validation and settings management library for Python. It enforces type hints at runtime and provides user-friendly errors when data is invalid.
1. Why do we need Pydantic?

In standard Python, type hinting is merely a suggestion. It does not stop "bad" data from entering your functions.
The Problem: Type Mismatch & Logic Errors
Python

def insert_patients(name: str, age: int):
    print(f"Name: {name}, Age: {age}")

# Python allows this! 'twenty' is a string, and age -5 is logically impossible.
insert_patients("Shaheer", "twenty") 

Without Pydantic, you must write endless Boilerplate Code (manual if/else checks) to ensure data integrity. Pydantic automates this.
2. The Pydantic Workflow (3-Step Process)

    Define the Schema (Model): Create a class inheriting from BaseModel. Define your fields, types, and constraints (e.g., age > 0).

    Instantiate & Validate: Pass your raw data (usually a dictionary) into the model using dictionary unpacking (**data). Pydantic checks types and constraints immediately.

    Consume Clean Data: Pass the validated object into your functions. The function no longer needs to check types; it can trust the object.

3. Advanced Validation Features

Pydantic offers specialized types and the Field class to add metadata and stricter rules.
Key Definitions:

    Annotated: Used to attach metadata (like Field) to a type without changing the type itself.

    Field: Used to customize validation (e.g., max_length, gt for greater than) and add documentation (description, examples).

    Strict Mode: By default, Pydantic tries to convert data (e.g., "75" becomes 75). Using strict=True prevents this "coercion."

Detailed Implementation
Python

from pydantic import BaseModel, EmailStr, Field, HttpUrl
from typing import List, Dict, Optional, Annotated

class Patient(BaseModel):
    # Annotated combines the type (str) with specific constraints
    name: Annotated[str, Field(max_length=50, description="Patient's full name", examples=["Shaheer"])]
    
    # EmailStr automatically validates email formatting
    email: EmailStr 
    
    age: int
    
    # weight must be > 0. strict=True means it won't convert "75.2" (string) to 75.2 (float)
    weight: Annotated[float, Field(gt=0, strict=True)] 
    
    # Optional field with a default value
    married: Optional[bool] = Field(default=None, description="Marital status")
    
    # List validation: must not exceed 5 items
    allergies: Optional[List[str]] = Field(default=[], max_length=5)
    
    # Dictionary validation: Key and Value must both be strings
    contact_detail: Dict[str, str]

# --- Usage ---
patient_info = {
    'name': 'Shaheer', 
    "email": "abc@gmail.com", 
    'age': 10, 
    'weight': 75.2, 
    'allergies': ["pollen"], 
    "contact_detail": {"phone": "03101234567"}
}

# Step 2: Object Creation & Validation
patient1 = Patient(**patient_info)

# Step 3: Use in Function
def insert_patient_to_db(patient: Patient):
    print(f"Adding {patient.name} to database...")
    if patient.allergies:
        print(f"Alert: Patient has {len(patient.allergies)} allergies.")

insert_patient_to_db(patient1)

4. Nested Models

For complex data, you can use one BaseModel as a field type inside another. This creates a clear hierarchy.
Python

class Address(BaseModel):
    city: str
    state: str
    pin: int 

class PatientRecord(BaseModel):
    name: str
    age: int
    address: Address # This links the two models

# Creating nested data
addr = {'city': 'Khanewal', 'state': 'Punjab', 'pin': 4324}
pat_data = {'name': 'Shaheer', 'age': 12, 'address': addr}

patient_obj = PatientRecord(**pat_data)
print(patient_obj.address.city) # Accessing nested data: Khanewal

5. Serialization (Exporting Data)

Serialization is converting your Pydantic object back into a format useful for APIs (FastAPI) or Databases.
Method	Output Type	Description
model_dump()	dict	Converts the object to a Python dictionary.
model_dump_json()	str	Converts the object to a JSON string.
Selection & Filtering

You can use include or exclude to control which data is exported:
Python

# Only export name and age
subset = patient1.model_dump(include={'name', 'age'})

# Export as JSON string
json_string = patient1.model_dump_json()